In [ ]:
import torch
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import random

from zipfile import ZipFile
file_path_zip = '/content/seg_train.zip'
with ZipFile(file_path_zip, 'r') as zip:
  zip.extractall()
  print('Done')

data = []

def create_training_data():
  file_path = '/content/seg_train'
  types = ['buildings','forest','glacier','mountain','sea','street']
  for category in types:
    class_num = types.index(category)
    path = os.path.join(file_path,category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img), 0)
      if img_array.shape == (150,150):
        img_array = np.reshape(img_array,22500)
        data.append([img_array,class_num])
        #data = list(data)

create_training_data()
data = np.asarray(data)
random.shuffle(data)
x = data[:,0]
print(x.shape)
y = torch.zeros(13986,6)
x = x.tolist()
Y = data[:,1]
Y = Y.tolist()

x = torch.Tensor(x)
x = x/255
y_data = torch.Tensor(Y)

for n in range(13986):
  if y_data[n] == 5:
    y[n,5] += 1
  elif y_data[n] == 4:
    y[n,4] += 1
  elif y_data[n] == 3:
    y[n,3] += 1
  elif y_data[n] == 2:
    y[n,2] += 1
  elif y_data[n] == 1:
    y[n,1] += 1
  elif y_data[n] == 0:
    y[n,0] += 1
print(y)

Done


In [120]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import random


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=4321)

class NeuralNetwork(nn.Module):
  def __init__(self,n_input,n_hidden_1,n_hidden_2,n_output):
    super(NeuralNetwork,self).__init__()
    self.l1 = nn.Linear(n_input,n_hidden_1)
    self.softmax = nn.Softmax()
    self.l2 = nn.Linear(n_hidden_1,n_hidden_2)
    self.l3 = nn.Linear(n_hidden_2,n_output)

  def forward(self,X):
    out = self.l1(X)
    out = self.softmax(out)
    out = self.l2(out)
    out = self.softmax(out)
    out = self.l3(out)
    out = self.softmax(out)
    return out

input_features = 22500
hidden_feature_1 = 128
hidden_feature_2 = 128
output_features = 6
model = NeuralNetwork(input_features,hidden_feature_1,hidden_feature_2,output_features)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005
optimiser = torch.optim.Adam(model.parameters(),lr=learning_rate)
iterations = 100000

for epoch in range(iterations):
  y_predicted = model(x_train)
  loss = criterion(y_predicted,y_train)
  loss.backward()
  optimiser.step()
  print('epoch: ' and epoch)
  if epoch == iterations:
    print('Training Complete')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0


KeyboardInterrupt: ignored